# LAB2 Artificial Intelligence for Autonomous Systems

First step is to convert input image to grayscale

In [1]:
# If note install, install pillow library
!pip install Pillow
!pip install matplotlib
!pip install opencv-python

In [3]:
from PIL import Image
import os

def convert_to_grayscale(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        input_path = os.path.join(input_dir, filename)

        if os.path.isfile(input_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            with Image.open(input_path) as img:
                grayscale_img =  img.convert('L')

                output_path = os.path.join(output_dir, filename)

                grayscale_img.save(output_path)
                print(f"Converted {filename} to grayscale and saved as {output_path}")

input_directory = "./dataset/base_input"
output_directory = "./dataset/grayscale_input"

convert_to_grayscale(input_directory, output_directory)


Converted photo_23_2024-03-19_17-10-36-copy-0.jpg to grayscale and saved as ./dataset/grayscale_input/photo_23_2024-03-19_17-10-36-copy-0.jpg
Converted photo_1_2024-03-19_17-10-36.jpg to grayscale and saved as ./dataset/grayscale_input/photo_1_2024-03-19_17-10-36.jpg
Converted photo_17_2024-03-19_17-10-36.jpg to grayscale and saved as ./dataset/grayscale_input/photo_17_2024-03-19_17-10-36.jpg
Converted photo_19_2024-03-19_17-10-36.jpg to grayscale and saved as ./dataset/grayscale_input/photo_19_2024-03-19_17-10-36.jpg
Converted photo_9_2024-03-19_17-10-36.jpg to grayscale and saved as ./dataset/grayscale_input/photo_9_2024-03-19_17-10-36.jpg
Converted photo_2_2024-03-19_17-10-36.jpg to grayscale and saved as ./dataset/grayscale_input/photo_2_2024-03-19_17-10-36.jpg
Converted photo_11_2024-03-19_17-10-36.jpg to grayscale and saved as ./dataset/grayscale_input/photo_11_2024-03-19_17-10-36.jpg
Converted photo_16_2024-03-19_17-10-36.jpg to grayscale and saved as ./dataset/grayscale_input/p

Now I'll apply 4 random rotation to each images. This step can be ignored if it doesn't contribute to improve the accuracy of our model

In [16]:
import random

def rotate_images(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        input_path = os.path.join(input_dir, filename)

        if os.path.isfile(input_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            with Image.open(input_path) as img:
                for i in range(4):
                    angle = random.randint(0, 360)
                    
                    rotated_img = img.rotate(angle)
                    
                    output_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_{angle}deg.jpg")
                    
                    rotated_img.save(output_path)
                    print(f"Applied {angle} degree rotation to {filename} and saved as {output_path}")

input_directory = "./dataset/grayscale_input"
output_directory = "./dataset/extended_set"

rotate_images(input_directory, output_directory)

Applied 235 degree rotation to photo_23_2024-03-19_17-10-36-copy-0.jpg and saved as ./dataset/extended_set/photo_23_2024-03-19_17-10-36-copy-0_235deg.jpg
Applied 147 degree rotation to photo_23_2024-03-19_17-10-36-copy-0.jpg and saved as ./dataset/extended_set/photo_23_2024-03-19_17-10-36-copy-0_147deg.jpg
Applied 229 degree rotation to photo_23_2024-03-19_17-10-36-copy-0.jpg and saved as ./dataset/extended_set/photo_23_2024-03-19_17-10-36-copy-0_229deg.jpg
Applied 59 degree rotation to photo_23_2024-03-19_17-10-36-copy-0.jpg and saved as ./dataset/extended_set/photo_23_2024-03-19_17-10-36-copy-0_59deg.jpg
Applied 136 degree rotation to photo_1_2024-03-19_17-10-36.jpg and saved as ./dataset/extended_set/photo_1_2024-03-19_17-10-36_136deg.jpg
Applied 263 degree rotation to photo_1_2024-03-19_17-10-36.jpg and saved as ./dataset/extended_set/photo_1_2024-03-19_17-10-36_263deg.jpg
Applied 256 degree rotation to photo_1_2024-03-19_17-10-36.jpg and saved as ./dataset/extended_set/photo_1_202

## Determine the line on each image 

This step will allow us to train the model we need some image with corresponding white line or vector

[Lane detection](https://medium.com/@mrhwick/simple-lane-detection-with-opencv-bfeb6ae54ec0)

1. Canny edge detection on the image to detect every line
2. Crop each image to clean the part where there's no chance to have a line.
3. Hough line transform in order to bring out line created by nearest pixel going in a same direction
4. Apply mask with the detected lines on the original picture

In [32]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2

height = width = 200
horizontal_ratio = 0.2

lower_threshold = 250
upper_threshold = 350

region_of_interest_vertices = [
    (0, height),
    (width / 2, height / 2 - (horizontal_ratio * height)),
    (width, height),
]

# Croppe region of interest
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)

    match_mask_color = 255

    cv2.fillPoly(mask, vertices, match_mask_color)

    masked_image = cv2.bitwise_and(img, mask)

    return masked_image

# Draw vector director on image
def draw_lines(img, lines, color=[255,0,0], thickness=3):
    if lines is None:
        return
    
    img = np.copy(img)
    
    line_img = np.zeros(
        (img.shape[0],
        img.shape[1]),
        dtype=np.uint8,
    )

    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_img, (x1 ,y1), (x2, y2), color, thickness)
    
    cv2.addWeighted(img, 0.8, line_img, 1.0, 0.0, img)

    return img

# Get the longest vector
def longest_vector(vector_list):
    max_length = 0
    longest_vector = None
    
    for vector in vector_list:
        x1, y1, x2, y2 = vector[0]
        length = ((x2 - x1)**2 + (y2 - y1)**2) ** 0.5
        if length > max_length:
            max_length = length
            longest_vector = vector
    
    return longest_vector

# Annotate imges
def img_lane_annotate(img):
    # Edge detection
    cannyed_image = cv2.Canny(img, lower_threshold, upper_threshold)

    # Cropped useless artifact
    cropped_image = region_of_interest(
        cannyed_image,
        np.array([region_of_interest_vertices], np.int32),
    )

    # Determine guidelines
    lines = []
    lines = cv2.HoughLinesP(
        cropped_image,
        rho=6,
        theta=np.pi / 60,
        threshold=10,
        lines=np.array([]),
        minLineLength=25,
        maxLineGap=10
    )

    print(lines)

    if lines is None:
        return img
    
    return draw_lines(img, line) 

image = mpimg.imread('./dataset/grayscale_input/photo_1_2024-03-19_17-10-36.jpg')

annotated_img = img_lane_annotate(image)

plt.figure()
plt.imshow(annotated_img)
plt.show()



[[ 67 141  67 110]]


TypeError: cannot unpack non-iterable numpy.int32 object

### Annotate every pictures in grayscale

In [25]:
def annotate_images(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Clean output dir
    for filename in os.listdir(output_dir):
        file_path = os.path.join(output_dir, filename)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

    # Annotate every image
    for idx, filename in enumerate(os.listdir(input_dir)):
        input_path = os.path.join(input_dir, filename)

        if os.path.isfile(input_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            img = mpimg.imread(input_path)
            annotated_img = img_lane_annotate(img)

            output_filename = f"image{idx}.jpg"
            output_path = os.path.join(output_dir, output_filename)

            mpimg.imsave(output_path, annotated_img)

input_directory = "./dataset/grayscale_input"
output_directory = "./output/line_annotated"

annotate_images(input_directory, output_directory)

[[[104 186 124 134]]

 [[112 143 127  97]]

 [[ 90 187 108 152]]]
[[[ 67 141  67 110]]]
[[[ 98 168 120 117]]

 [[104 131 118  95]]

 [[114 134 126  97]]]
[[[ 44 138 120  89]]

 [[ 68 124 114  94]]

 [[ 85 121 123  93]]]
[[[ 92  72 136 140]]

 [[ 93  86 122 132]]

 [[104 105 123 134]]]
[[[ 66 106  84 137]]]
None
[[[ 78 155 101 119]]

 [[150 156 175 165]]

 [[ 84 104  96 129]]

 [[ 81  86 122 158]]

 [[ 56 199  94 124]]

 [[ 65 184  81 155]]

 [[107 194 116 167]]]
[[[ 84 107 124 144]]

 [[101 132 125 159]]]
[[[ 67 106 124 177]]

 [[ 69 104 106 141]]

 [[ 68 108  97  89]]

 [[ 67 107 102 150]]]
[[[ 19 186  51 170]]

 [[ 44 141  84 144]]

 [[127 152 131 110]]]
[[[ 75  95  80 122]]]
[[[104 104 160 154]]

 [[ 84  82 160 143]]

 [[146 151 169 177]]

 [[123 181 124 153]]

 [[125 199 125 159]]]
[[[ 67 129  71 102]]]
[[[ 56 121  67 147]]]
[[[100 127 120 100]]]
[[[ 79  88 138 114]]

 [[ 92 101 136 121]]]
[[[ 60 167 111 121]]

 [[ 84  82 113 148]]

 [[125 199 154 142]]

 [[ 89 175 132 132]]

 [[ 8